In [25]:
# Tratamiento de datos
# ------------------------------------------------------------------------------
import numpy as np
import pandas as pd

# Gráficos
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Modelado y evaluación
# ------------------------------------------------------------------------------
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import tree
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV

# Barra de progreso de un proceso
# ------------------------------------------------------------------------------
from tqdm import tqdm

# Configuración warnings
# ------------------------------------------------------------------------------
import warnings
warnings.filterwarnings('once')

# Pair Programming Decision Tree

En el pair programming de hoy debéis usar el csv que guardastéis cuando hicistéis el pairprgramming de codificicación (este csv debería tener las variables estadandarizas).

En pairprogramming anteriores ajustastéis vuestro datos a una regresión lineal y a un Decision Tree. El objetivo de hoy es:
- Ajustar el modelo a un Random Forest
- Extraer las métricas
- Debatid entre vosotras que modelo es mejor y por qué (basándose en las métricas)

In [26]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [27]:
df = pd.read_csv('datos/soci_econ_country_profiles_v7.csv', index_col=0)
df.head()

,Region,GDP_per_capita,Employment_Agriculture,Employment_Services,Agricultural_production_index,Urban_population,Urban_population_growth_rate,Fertility_rate_total,Infant_mortality_rate,Seats_held_by_women_in_national_parliaments,Individuals_using_the_Internet,Energy_production_primary,Energy_supply_per_capita,Quality_Of_Life_Index,Purchasing_Power_Index,Safety_Index,Health_Care_Index,Property_price_to_income_ratio,Traffic_commute_time_index,Pollution_index,Climate_index,Affordability_Index,Cost_Of_Living_Index,Consumer_price_index,Current_health_expenditure,Human_capital_index,Inflation_consumer_prices,Life_expectancy_at_birth_total,Labour_force_participation,Pop_using_improved_drinking_water_urban,Pop_using_improved_drinking_water_rural,Population_age_distribution_0_14,Population_age_distribution_60_mas,Education_Secondary_gross_enrol_ratio,Education_Tertiary_gross_enrol_ratio
country,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Argentina,1.84,-0.431499,-0.793293,0.635685,0.451515,1.176545,-0.146278,0.816818,0.390527,1.373669,-0.170271,-0.433123,-0.072257,139.59,-0.480096,-1.843490,0.739348,0.085614,0.253993,0.085118,1.031920,-1.242902,-0.124619,-0.375346,0.595817,-0.661735,-0.179334,-0.356054,0.526225,0.213117,0.576120,0.983668,-0.481642,0.119029,0.928674
Australia,2.30,1.358804,-0.729591,0.874487,-0.051041,1.039907,0.271660,0.093270,-0.510644,0.436646,1.674834,0.729295,0.579071,176.54,1.024540,-0.270524,0.839394,-0.461542,-0.207625,-1.254411,0.556493,0.080872,1.230619,-0.300239,0.626171,1.061166,-0.270570,1.069555,0.052782,0.675804,0.681506,-0.044663,0.256053,1.643795,1.272043
Austria,2.30,1.006731,-0.547584,0.365043,-0.239499,-0.292313,-0.647804,-0.811165,-0.565818,0.611189,-0.538225,0.186268,0.579071,190.37,0.807519,1.532822,1.540764,-0.380339,-1.420448,-1.379278,0.273045,0.414798,0.789648,-0.345498,1.068892,0.971432,-0.238703,0.870393,-0.574209,0.675804,0.681506,-0.898700,0.796151,-0.222299,0.863591
Belarus,1.84,-0.860425,-0.101668,-0.534443,0.639974,0.316865,-0.982155,-0.449391,-0.538231,0.969463,-0.786194,-0.119185,0.513938,119.23,-1.239326,1.298550,-1.318426,1.373269,-1.221178,-0.552731,-0.845325,-1.397939,-1.028272,1.151509,-0.593477,-0.907608,0.710551,-0.877799,-0.318294,-0.042578,0.383944,-0.445537,0.295572,0.131486,1.157587
Belgium,2.30,0.819859,-0.866096,0.993887,-0.239499,1.523832,-0.564217,-0.087617,-0.547427,1.290990,-0.754198,0.508691,0.579071,160.52,0.647171,-0.433191,1.331195,-0.902361,0.011715,-0.120354,0.619511,1.070722,0.974914,-0.368860,1.047345,0.648387,-0.227962,0.835214,-1.457115,0.614924,0.644311,-0.375820,0.730285,3.118732,0.569595


In [28]:
X = df.drop(['Quality_Of_Life_Index'], axis=1)
y = df ['Quality_Of_Life_Index']

In [29]:
# dividimos los datos en datos de entrenamiento y datos de prueba

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 37)

In [30]:
# función para sacar las métricas a un dataframe

def metricas(y_test, y_train, y_test_pred, y_train_pred, tipo_modelo):
    
    
    resultados = {'MAE': [mean_absolute_error(y_test, y_test_pred), mean_absolute_error(y_train, y_train_pred)],
                'MSE': [mean_squared_error(y_test, y_test_pred), mean_squared_error(y_train, y_train_pred)],
                'RMSE': [np.sqrt(mean_squared_error(y_test, y_test_pred)), np.sqrt(mean_squared_error(y_train, y_train_pred))],
                'R2':  [r2_score(y_test, y_test_pred), r2_score(y_train, y_train_pred)],
                 "set": ["test", "train"]}
    df = pd.DataFrame(resultados)
    df["modelo"] = tipo_modelo
    return df

In [31]:
# ajustamos los parámetros según el modelo de decision tree realizado en el pair anterior
param = {"max_depth": [3,4,5], 
        "max_features": [3,4,5],
        "min_samples_split": [5, 10, 20],
        "min_samples_leaf": [5, 10, 20]}

In [32]:
gs_rf = GridSearchCV(
            estimator=RandomForestRegressor(), # tipo de modelo que queremos hacer
            param_grid= param, # que hiperparámetros queremos que testee
            cv=10, # crossvalidation que aprendimos en la lección de regresión lineal intro. 
            verbose=-1, # para que no nos printee ningún mensaje en pantalla
            return_train_score = True, # para que nos devuelva el valor de las métricas de set de datos de entrenamiento
            scoring="neg_mean_squared_error") # la métrica que queremos que nos devuelva

In [33]:
gs_rf.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [3, 4, 5], 'max_features': [3, 4, 5],
                         'min_samples_leaf': [5, 10, 20],
                         'min_samples_split': [5, 10, 20]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=-1)

In [34]:
bosque = gs_rf.best_estimator_
bosque

RandomForestRegressor(max_depth=3, max_features=5, min_samples_leaf=5,
                      min_samples_split=10)

In [35]:
y_pred_test_rf = bosque.predict(x_test)
y_pred_train_rf = bosque.predict(x_train)

In [36]:
dt_results = metricas(y_test, y_train, y_pred_test_rf, y_pred_train_rf, "Random Forest")
dt_results

,MAE,MSE,RMSE,R2,set,modelo
0,13.210086,423.353216,20.575549,0.659824,test,Random Forest
1,10.394530,194.675933,13.952632,0.874387,train,Random Forest


Observamos que las métricas mejoran respecto al modelo de DT.

In [37]:
train_df = pd.DataFrame({'Real': y_train, 'Predicted': y_pred_train_rf, 'Set': ['Train']*len(y_train)})
test_df = pd.DataFrame({'Real': y_test, 'Predicted': y_pred_test_rf, 'Set': ['Test']*len(y_test)})
resultados = pd.concat([train_df,test_df], axis = 0)

In [38]:
resultados

,Real,Predicted,Set
country,,,
Romania,143.04,127.383576,Train
Egypt,91.81,98.573672,Train
New Zealand,184.74,172.628659,Train
Belgium,160.52,167.658223,Train
Vietnam,36.12,84.726007,Train
Colombia,115.38,102.967065,Train
Lithuania,130.28,139.189525,Train
Sweden,172.74,172.189969,Train
Germany,189.74,174.630438,Train


In [39]:
df_DT_LR = pd.read_csv("datos/resultados_RL.csv", index_col=0)
df_DT_LR.head()

,MAE,MSE,RMSE,R2,set,modelo
0,0.010049,0.000130,0.011398,1.000000,test,Linear Regresion
1,0.001999,0.000008,0.002763,1.000000,train,LinearRegression
0,17.704615,794.803877,28.192266,0.361354,test,Decision Tree I
1,0.000000,0.000000,0.000000,1.000000,train,Decision Tree I
0,16.139883,803.812824,28.351593,0.354115,test,Decision tree II


In [40]:
df_RF_DT_LR_results = pd.concat([df_DT_LR, dt_results], axis = 0)
df_RF_DT_LR_results

,MAE,MSE,RMSE,R2,set,modelo
0,0.010049,0.000130,0.011398,1.000000,test,Linear Regresion
1,0.001999,0.000008,0.002763,1.000000,train,LinearRegression
0,17.704615,794.803877,28.192266,0.361354,test,Decision Tree I
1,0.000000,0.000000,0.000000,1.000000,train,Decision Tree I
0,16.139883,803.812824,28.351593,0.354115,test,Decision tree II
1,11.612891,270.326345,16.441604,0.825574,train,Decision tree II
0,13.210086,423.353216,20.575549,0.659824,test,Random Forest
1,10.394530,194.675933,13.952632,0.874387,train,Random Forest


En comparación, los resultados del modelo Random Forest son los mejores para nuestro conjunto de datos; aunque aún se observa overfitting, todas las métricas mejoran respecto a los otros modelos.

In [44]:
# intentamos ajustar los hiperparámetros para mejorar el modelo
param2 = {"max_depth": [2,3,4], 
        "max_features": [3, 5, 7],
        "min_samples_split": [2, 5, 10],
        "min_samples_leaf": [5, 10, 20]} 

In [45]:
gs_rf2 = GridSearchCV(
            estimator=RandomForestRegressor(), # tipo de modelo que queremos hacer
            param_grid= param2, # que hiperparámetros queremos que testee
            cv=10, # crossvalidation que aprendimos en la lección de regresión lineal intro. 
            verbose=-1, # para que no nos printee ningún mensaje en pantalla
            return_train_score = True, # para que nos devuelva el valor de las métricas de set de datos de entrenamiento
            scoring="neg_mean_squared_error") # la métrica que queremos que nos devuelva

In [46]:
gs_rf2.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [2, 3, 4], 'max_features': [3, 5, 7],
                         'min_samples_leaf': [5, 10, 20],
                         'min_samples_split': [2, 5, 10]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=-1)

In [47]:
bosque2 = gs_rf2.best_estimator_
bosque2

RandomForestRegressor(max_depth=3, max_features=7, min_samples_leaf=5,
                      min_samples_split=10)

In [48]:
y_pred_test_rf2 = bosque2.predict(x_test)
y_pred_train_rf2 = bosque2.predict(x_train)

In [49]:
dt_results2 = metricas(y_test, y_train, y_pred_test_rf2, y_pred_train_rf2, "Random Forest")
dt_results2

,MAE,MSE,RMSE,R2,set,modelo
0,13.592581,440.356386,20.984670,0.646162,test,Random Forest
1,9.587942,173.971589,13.189829,0.887746,train,Random Forest


Al observar los resultados de este nuevo modelo de Random Forest no apreciamos una mejora respecto al anterior. Por tanto, creemos que para nuestro conjunto de datos el modelo que mejor ajusta es el primer modelo de Random Forest que hemos creado en este pair.